In [ ]:
from collections import deque

# Hàm kiểm tra xem trạng thái hiện tại có phải trạng thái đích chưa
def is_goal(state, goal):
    return state == goal

# Hàm tìm các trạng thái con từ trạng thái hiện tại
def get_neighbors(state):
    neighbors = []
    n = int(len(state) ** 0.5)  # Tính kích thước lưới (ví dụ 3x3 nếu 8-puzzle)
    zero_index = state.index(0)  # Tìm vị trí ô trống (ô số 0)
    x, y = divmod(zero_index, n)  # Tính hàng (x) và cột (y)

    # Các hướng di chuyển: lên, xuống, trái, phải
    moves = [(-1,0), (1,0), (0,-1), (0,1)]

    for dx, dy in moves:
        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < n and 0 <= new_y < n:
            new_zero_index = new_x * n + new_y
            new_state = list(state)
            # Đổi chỗ 0 với số kế bên
            new_state[zero_index], new_state[new_zero_index] = new_state[new_zero_index], new_state[zero_index]
            neighbors.append(tuple(new_state))

    return neighbors

# Hàm chính BFS
def bfs(start, goal):
    MO = deque()
    MO.append((start, [start]))  # Mỗi phần tử lưu (trạng thái hiện tại, đường đi từ đầu tới đó)
    DONG = set()
    DONG.add(start)

    while MO:
        current_state, path = MO.popleft()

        if is_goal(current_state, goal):
            return path  # Trả về đường đi

        for neighbor in get_neighbors(current_state):
            if neighbor not in DONG:
                DONG.add(neighbor)
                MO.append((neighbor, path + [neighbor]))

    return None  # Không tìm thấy lời giải

# Ví dụ sử dụng
if __name__ == "__main__":
    start = (0, 4, 2,
             6, 5, 1,
             8, 7, 3)

    goal = (1, 2, 3,
            4, 5, 6,
            7, 8, 0)

    path = bfs(start, goal)

    if path:
        print(f"Số bước: {len(path) - 1}")
        for state in path:
            for i in range(0, len(state), 3):
                print(state[i:i+3])
            print("---------")
    else:
        print("Không tìm thấy lời giải.")

Số bước: 22
(0, 4, 2)
(6, 5, 1)
(8, 7, 3)
---------
(6, 4, 2)
(0, 5, 1)
(8, 7, 3)
---------
(6, 4, 2)
(5, 0, 1)
(8, 7, 3)
---------
(6, 4, 2)
(5, 7, 1)
(8, 0, 3)
---------
(6, 4, 2)
(5, 7, 1)
(0, 8, 3)
---------
(6, 4, 2)
(0, 7, 1)
(5, 8, 3)
---------
(6, 4, 2)
(7, 0, 1)
(5, 8, 3)
---------
(6, 4, 2)
(7, 1, 0)
(5, 8, 3)
---------
(6, 4, 2)
(7, 1, 3)
(5, 8, 0)
---------
(6, 4, 2)
(7, 1, 3)
(5, 0, 8)
---------
(6, 4, 2)
(7, 1, 3)
(0, 5, 8)
---------
(6, 4, 2)
(0, 1, 3)
(7, 5, 8)
---------
(0, 4, 2)
(6, 1, 3)
(7, 5, 8)
---------
(4, 0, 2)
(6, 1, 3)
(7, 5, 8)
---------
(4, 1, 2)
(6, 0, 3)
(7, 5, 8)
---------
(4, 1, 2)
(0, 6, 3)
(7, 5, 8)
---------
(0, 1, 2)
(4, 6, 3)
(7, 5, 8)
---------
(1, 0, 2)
(4, 6, 3)
(7, 5, 8)
---------
(1, 2, 0)
(4, 6, 3)
(7, 5, 8)
---------
(1, 2, 3)
(4, 6, 0)
(7, 5, 8)
---------
(1, 2, 3)
(4, 0, 6)
(7, 5, 8)
---------
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)
---------
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)
---------


In [ ]:
import heapq
import math
import time

# Heuristic 1: Tổng số ô sai vị trí
def heuristic_1(state, goal):
    return sum(1 for i in range(len(state)) if state[i] != 0 and state[i] != goal[i])

# Heuristic 2: Tổng khoảng cách để đưa các ô về đúng vị trí
def heuristic_2(state, goal):
    n = int(len(state) ** 0.5)
    distance = 0
    for i in range(len(state)):
        if state[i] == 0:
            continue
        goal_index = goal.index(state[i])
        x1, y1 = divmod(i, n)
        x2, y2 = divmod(goal_index, n)
        distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

# Heuristic 3: Tổng khoảng cách Euler của các ô với vị trí đích
def heuristic_3(state, goal):
    n = int(len(state) ** 0.5)
    distance = 0
    for i in range(len(state)):
        if state[i] == 0:
            continue
        goal_index = goal.index(state[i])
        x1, y1 = divmod(i, n)
        x2, y2 = divmod(goal_index, n)
        distance += math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
    return distance

# Heuristic 4: Tổng số ô sai hàng và số ô sai cột
def heuristic_4(state, goal):
    n = int(len(state) ** 0.5)
    h = 0
    for i in range(len(state)):
        if state[i] == 0:
            continue
        goal_index = goal.index(state[i])
        x1, y1 = divmod(i, n)
        x2, y2 = divmod(goal_index, n)
        if x1 != x2:
            h += 1
        if y1 != y2:
            h += 1
    return h

# Heuristic 5: Tổng khoảng cách để đưa các ô về đúng vị trí + số ô xung đột tuyến tính
def heuristic_5(state, goal):
    n = int(len(state) ** 0.5)
    manhattan = heuristic_2(state, goal)
    linear_conflict = 0

    # Row conflicts
    for row in range(n):
        row_start = row * n
        for i in range(n):
            for j in range(i + 1, n):
                a = state[row_start + i]
                b = state[row_start + j]
                if a != 0 and b != 0:
                    a_goal = goal.index(a)
                    b_goal = goal.index(b)
                    if a_goal // n == row and b_goal // n == row:
                        if a_goal > b_goal:
                            linear_conflict += 1

    # Column conflicts
    for col in range(n):
        for i in range(n):
            for j in range(i + 1, n):
                a = state[i * n + col]
                b = state[j * n + col]
                if a != 0 and b != 0:
                    a_goal = goal.index(a)
                    b_goal = goal.index(b)
                    if a_goal % n == col and b_goal % n == col:
                        if a_goal > b_goal:
                            linear_conflict += 1

    return manhattan + 2 * linear_conflict

# Heuristic 6: heuristic 5 + số ô không thể về đích
def heuristic_6(state, goal):
    n = int(len(state) ** 0.5)
    h = heuristic_5(state, goal)

    blocked_tiles = 0
    for i in range(len(state)):
        if state[i] == 0 or state[i] == goal[i]:
            continue
        x, y = divmod(i, n)
        surrounded = True
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < n and 0 <= ny < n:
                ni = nx * n + ny
                if state[ni] != goal[ni]:
                    surrounded = False
                    break
        if surrounded:
            blocked_tiles += 1

    return h + blocked_tiles

# Lấy hàm heuristic theo type
def get_heuristic(state, goal, type):
    if type == 1:
        return heuristic_1(state, goal)
    elif type == 2:
        return heuristic_2(state, goal)
    elif type == 3:
        return heuristic_3(state, goal)
    elif type == 4:
        return heuristic_4(state, goal)
    elif type == 5:
        return heuristic_5(state, goal)
    elif type == 6:
        return heuristic_6(state, goal)
    else:
        raise ValueError("Invalid heuristic type")

# Sinh trạng thái lân cận
def get_neighbors(state):
    n = int(len(state) ** 0.5)
    zero = state.index(0)
    x, y = divmod(zero, n)
    neighbors = []

    directions = [(-1,0),(1,0),(0,-1),(0,1)]
    for dx, dy in directions:
        nx, ny = x + dx, y + dy
        if 0 <= nx < n and 0 <= ny < n:
            nz = nx * n + ny
            new_state = list(state)
            new_state[zero], new_state[nz] = new_state[nz], new_state[zero]
            neighbors.append(tuple(new_state))

    return neighbors

# Kiểm tra trạng thái có giải được không
def is_solvable(puzzle):
    n = int(len(puzzle) ** 0.5)
    inv_count = 0
    arr = [x for x in puzzle if x != 0]

    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            if arr[i] > arr[j]:
                inv_count += 1

    if n % 2 == 1:
        return inv_count % 2 == 0
    else:
        row_blank = puzzle.index(0) // n
        return (inv_count + row_blank) % 2 == 1

# Giải thuật A*
def a_star(start, goal, heuristic_type=1):
    if not is_solvable(start):
        print("Trạng thái KHÔNG thể giải được!")
        return None

    open_set = []
    heapq.heappush(open_set, (0, start))
    came_from = {}
    g_score = {start: 0}
    visited = set()

    while open_set:
        _, current = heapq.heappop(open_set)
        visited.add(current)

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        for neighbor in get_neighbors(current):
            tentative_g = g_score[current] + 1
            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f = tentative_g + get_heuristic(neighbor, goal, heuristic_type)
                heapq.heappush(open_set, (f, neighbor))

    return None

# In trạng thái
def print_state(state):
    n = int(len(state) ** 0.5)
    for i in range(0, len(state), n):
        print(state[i:i+n])
    print("---")

# MAIN
if __name__ == "__main__":
    start = (0, 4, 2,
             6, 5, 1,
             8, 7, 3)

    goal = (1, 2, 3,
            4, 5, 6,
            7, 8, 0)

    for h in range(1, 7):
        print(f"\n===== Heuristic {h} =====")
        t0 = time.time()
        path = a_star(start, goal, heuristic_type=h)
        t1 = time.time()

        if path:
            print(f"Tìm thấy lời giải trong {len(path)-1} bước, thời gian: {t1 - t0:.4f} giây")
            for step in path:
                print_state(step)
        else:
            print("Không tìm thấy lời giải.")


===== Heuristic 1 =====
Tìm thấy lời giải trong 22 bước, thời gian: 0.0967 giây
(0, 4, 2)
(6, 5, 1)
(8, 7, 3)
---
(6, 4, 2)
(0, 5, 1)
(8, 7, 3)
---
(6, 4, 2)
(5, 0, 1)
(8, 7, 3)
---
(6, 4, 2)
(5, 7, 1)
(8, 0, 3)
---
(6, 4, 2)
(5, 7, 1)
(0, 8, 3)
---
(6, 4, 2)
(0, 7, 1)
(5, 8, 3)
---
(6, 4, 2)
(7, 0, 1)
(5, 8, 3)
---
(6, 4, 2)
(7, 1, 0)
(5, 8, 3)
---
(6, 4, 2)
(7, 1, 3)
(5, 8, 0)
---
(6, 4, 2)
(7, 1, 3)
(5, 0, 8)
---
(6, 4, 2)
(7, 1, 3)
(0, 5, 8)
---
(6, 4, 2)
(0, 1, 3)
(7, 5, 8)
---
(6, 4, 2)
(1, 0, 3)
(7, 5, 8)
---
(6, 0, 2)
(1, 4, 3)
(7, 5, 8)
---
(0, 6, 2)
(1, 4, 3)
(7, 5, 8)
---
(1, 6, 2)
(0, 4, 3)
(7, 5, 8)
---
(1, 6, 2)
(4, 0, 3)
(7, 5, 8)
---
(1, 0, 2)
(4, 6, 3)
(7, 5, 8)
---
(1, 2, 0)
(4, 6, 3)
(7, 5, 8)
---
(1, 2, 3)
(4, 6, 0)
(7, 5, 8)
---
(1, 2, 3)
(4, 0, 6)
(7, 5, 8)
---
(1, 2, 3)
(4, 5, 6)
(7, 0, 8)
---
(1, 2, 3)
(4, 5, 6)
(7, 8, 0)
---

===== Heuristic 2 =====
Tìm thấy lời giải trong 22 bước, thời gian: 0.0137 giây
(0, 4, 2)
(6, 5, 1)
(8, 7, 3)
---
(4, 0, 2)
(6, 5, 1)
(8